<a href="https://colab.research.google.com/github/Christheoneoneil/stat330_final/blob/model_fitting/model_fitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Relevant Imports


In [1]:
import stan
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np
import arviz as az
import seaborn as sns

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Preliminary Code Cleaning for Multinomial Logistic Regression

In [2]:
def read_data(file_name: str):
  """
  param filename: name of data file
  returns: pandas data frame
  """
  df = pd.read_csv(file_name, index_col = "Unnamed: 0")
  df.dropna(axis = 0, how="any", inplace=True)
  df = df.loc[:,~df.columns.duplicated()]
  return df

In [3]:
def prep_data(df: pd.DataFrame, key: str, unwanted_cols: list):
  """
  param df: data frame that needs to be formatted
  key: key value associated with provided dictionary
  unwanted_cols: columns that are ultimatly not needed for analysis
  returns: prept data frame for logit regression
  """
  from sklearn import preprocessing
  df_copy = df.copy()
  unnormed_cols = ["STUDWGT"]

  if key == "imputed":
    unnormed_cols = ['ACTComposite', 'SATMath', 
                     'SATVerbal', 'SATWriting'] + unnormed_cols
    df_copy.drop(columns=["Unnamed: 0.1"], inplace=True)

  normalizer = preprocessing.MinMaxScaler()
  normed_cols = normalizer.fit_transform(df[unnormed_cols])
  df_copy[unnormed_cols] = normed_cols
  df_copy.drop(columns=unwanted_cols, inplace=True)

  recode_vars = ["STRAT", "SELECTIVITY", "DOBYear"]
  for var in recode_vars:
    unencoded_list = list(df_copy[var].unique())
    encode_list = list(range(1, len(unencoded_list)+1))
    df_copy[var].replace(unencoded_list, encode_list, inplace=True)
  return(df_copy)

In [4]:
df_dict = {"imputed": read_data("final_frame_imputed.csv"),
           "non_imputed": read_data("final_frame_non_imputed.csv")}
unnecessary_cols = ["ACERECODE"]
for key, df in df_dict.items():
  df_dict[key] = prep_data(df, key, unnecessary_cols)

FileNotFoundError: ignored

#Setting Up Model

In [ ]:
def fit_model(model_code: str, X: np.array, Y: np.array, n: int, k: int, 
              flag_val: int):
  """
  param model_code: stan formatted code for model
  param X: nxk array of covariates 
  param Y: array of target values
  param n: number of rows 
  param k: number of covarites
  param flag_val: do predictive distrub flag value
  reutrns: stan sampler object
  """
  mod = stan.build(model_code,data={"X": X, "Y": Y, "n": n, "k": k, 
                               "do_prior_predictive": flag_val})
  samples = mod.sample()
  return samples


In [ ]:
multinomial_log_stan_code = """
data {
  int<lower=0> n;                // number of units
  int<lower=0> k;                // number of covariates
  matrix[n, k] X;            // covariates for each entry, including the intercept covariate
  int<lower=1,upper=5> Y[n];     // categorical

  int do_prior_predictive;
}
parameters {
  vector[k] beta;
  vector<lower=0>[k] lambda;
  real<lower=0> tau;
  real<lower=0> sigma;            // the coefficients
}

transformed parameters {
  vector[n] = mu;
  vector[n] eta;                // linear predictors
  eta = X * beta;
  mu = inv_logit(eta)
}
model {
  lambda ~ cauchy(0, 1);
  tau ~ cauchy(0, 1);
  for (i in 1:k)
    beta[i] ~ normal(0, lambda[i] * tau);
  if (do_prior_predictive != 1) {
    for (i in 1:n)
      Y[i] ~ categorical_logit_glm(mu);
  }
}
generated quantities {
  int<lower=1,upper=4> Y_tilde[n];
  for (i in 1:n)  
    Y_tilde[i] = categorical_logit_glm_rng(mu);  
}
"""

In [ ]:
target_col = "CHOICE"
prior_pred_dict = {}
for key, df in df_dict.items():
  df = df.sample(frac=0.20)
  covars = [cov for cov in list(df.columns) if cov not in target_col]
  X = np.array(df[covars])
  n = len(X)
  X = np.concatenate((np.ones((n, 1)), X), axis=1)
  Y = np.array(df[target_col], dtype="int64")
  k = X.shape[1]
  prior_pred_dict[key] = fit_model(multinomial_log_stan_code, X, Y, n, k, 1)